### Анализ зарплаты сотрудникам с помощью парсинга hh.ru

In [5]:
def normalize_salary(salary):
    # Курсы валют
    exchange_rates = {
        '₽': 1,       # Российский рубль
        '$': 90,      # Доллар США
        'Br': 30,     # Белорусский рубль
        '₸': 0.2,     # Казахстанский тенге
    }

    salary = salary.replace('\xa0', '').replace('\u2009', '')
    currency = salary[-1] if salary else '₽'
    salary_value = ''.join([char for char in salary if char.isdigit()])
    salary_value = int(salary_value) if salary_value else 0

    if currency in exchange_rates:
        return salary_value * exchange_rates[currency]
    else:
        return salary_value

In [20]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

data = []

search = 'преподаватель по IT разработке'

# Перебираем 10 страниц
for i in range(10):
    url = f"https://hh.ru/search/resume?area=113&exp_period=all_time&logic=normal&no_magic=true&order_by=relevance&ored_clusters=true&pos=full_text&search_period=30&text={search.replace(' ', '+')}&items_on_page=20&searchSessionId=56755fc9-4691-4c95-82d3-cfcbc864240a&page={i}"

    response = requests.get(url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')

        resumes = soup.find_all('div', class_='column-content--q3SfppwQANVUv38P')


        for resume in resumes:
            title = resume.find('span', class_='magritte-text___tkzIl_5-0-7')
            title = title.text.strip() if title else None
            salary = resume.find('div', class_='magritte-text___pbpft_3-0-31 magritte-text_style-primary___AQ7MW_3-0-31 magritte-text_typography-subtitle-2-semibold___3q3c-_3-0-31')
            salary = normalize_salary(salary.text.strip()) if salary else None

            data.append({
                'Специальность': title,
                'Зарплата': salary
            })

df = pd.DataFrame(data)
df

,Специальность,Зарплата
0,Middle Developer,180000.0
1,Проектный менеджер it,180000.0
2,Руководитель отдела разработки,350000.0
3,"Руководитель ЦТ (Руководитель IT), CDTO, CIO, ...",75000.0
4,Директор IT-проектов,NaN
...,...,...
195,Тестировщик ПО (Junior),NaN
196,Специалист HR,100000.0
197,Исполнительный директор,700000.0
198,Директор e-commerce,NaN


In [21]:
df = df.dropna(subset='Зарплата')
df

,Специальность,Зарплата
0,Middle Developer,180000.0
1,Проектный менеджер it,180000.0
2,Руководитель отдела разработки,350000.0
3,"Руководитель ЦТ (Руководитель IT), CDTO, CIO, ...",75000.0
7,Руководитель разработки | Технический директор...,350000.0
...,...,...
192,"Руководитель проекта, программист 1с",100000.0
193,"Product owner, project manager, trainer, consu...",1800.0
196,Специалист HR,100000.0
197,Исполнительный директор,700000.0


In [22]:
import plotly.express as px
fig = px.histogram(df, x='Зарплата', title='Зарплата преподавателей IT-курсов на основе резюме на hh.ru')
fig.update_xaxes(
    title_text='Зарплата'
)

fig.update_yaxes(
    title_text='Количество резюме'
)
fig.show()